In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import drive

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# ===============================
# IMPORTAN: NORMALISASI IMAGE
# ===============================
# Mean dan std ini adalah standar ImageNet.
# WAJIB digunakan jika memakai model pretrained (ResNet, VGG, dll)
mean = np.array([0.485, 0.456, 0.406])   # mean untuk channel R, G, B
std = np.array([0.229, 0.224, 0.225])    # standar deviasi untuk channel R, G, B


# ===============================
# DATA TRANSFORMATION PIPELINE
# ===============================
data_transforms = {
    'train': transforms.Compose([
        # Random crop ke ukuran 224x224
        # Berfungsi sebagai data augmentation
        transforms.RandomResizedCrop(224),

        # Flip horizontal secara acak
        # Membantu model lebih general (tidak overfitting)
        transforms.RandomHorizontalFlip(),

        # Konversi PIL Image → Tensor (C, H, W) dengan range [0, 1]
        transforms.ToTensor(),

        # Normalisasi berdasarkan mean & std ImageNet
        transforms.Normalize(mean, std)
    ]),

    'val' : transforms.Compose([
        # Resize sisi terpendek ke 256
        transforms.Resize(256),

        # Center crop 224x224 (TIDAK random)
        # Validasi harus konsisten, bukan acak
        transforms.CenterCrop(224),

        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}


# ===============================
# LOAD DATASET DARI FOLDER
# ===============================
# Struktur folder:
# hymenoptera_data/
# ├── train/
# │   ├── ants/
# │   └── bees/
# └── val/
#     ├── ants/
#     └── bees/
data_dir = "/content/drive/MyDrive/Triad/Kuliah/Machine Learning/src/hymenoptera_data"

# Dataset train dan validation
image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x),
        data_transforms[x]
    )
    for x in ['train', 'val']
}


# ===============================
# DATALOADER
# ===============================
# DataLoader bertugas:
# - batching
# - shuffle
# - parallel loading (num_workers)
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x],
        batch_size=4,
        shuffle=True,
        num_workers=0
    )
    for x in ['train', 'val']
}

# Jumlah data tiap fase
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# Nama kelas (diambil dari nama folder)
class_names = image_datasets['train'].classes
print(class_names)


# ===============================
# FUNGSI TRAINING & VALIDATION
# ===============================
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    # Simpan bobot terbaik berdasarkan akurasi validation
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Loop epoch
    for epoch in range(num_epochs):
        print(f'Epoch: {epoch}/{num_epochs - 1}')
        print('-'*10)

        # Loop untuk train dan validation
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # mode training (dropout aktif, batchnorm update)
            else:
                model.eval()   # mode evaluasi

            running_loss = 0.0
            running_corrects = 0

            # Loop batch
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Gradien hanya dihitung saat training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)         # forward pass
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backpropagation hanya saat training
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # Akumulasi loss & prediksi benar
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Update learning rate scheduler setelah 1 epoch training
            if phase == 'train':
                scheduler.step()

            # Hitung loss & akurasi per epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Simpan model terbaik berdasarkan validation accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # Informasi waktu training
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load bobot terbaik
    model.load_state_dict(best_model_wts)
    return model


# ===============================
# TRANSFER LEARNING: RESNET18
# ===============================
# Load model ResNet18 pretrained ImageNet
model = models.resnet18(pretrained=True)

# Bekukan semua parameter convolutional
# Artinya: feature extractor TIDAK dilatih ulang
for param in model.parameters():
    param.requires_grad = False

# Ambil jumlah fitur dari layer FC terakhir
num_ftrs = model.fc.in_features

# Ganti fully connected layer:
# dari 1000 kelas ImageNet → 2 kelas (ants, bees)
model.fc = nn.Linear(num_ftrs, 2)

# Pindahkan model ke GPU/CPU
model.to(device)


# ===============================
# LOSS, OPTIMIZER, SCHEDULER
# ===============================
criterion = nn.CrossEntropyLoss()              # loss untuk klasifikasi multi-class
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Scheduler: turunkan learning rate setiap 7 epoch
step_lr_scheduler = lr_scheduler.StepLR(
    optimizer,
    step_size=7,
    gamma=0.1
)

# ===============================
# TRAIN MODEL
# ===============================
model = train_model(
    model,
    criterion,
    optimizer,
    step_lr_scheduler,
    num_epochs=20
)


['ants', 'bees']
Epoch: 0/19
----------
train Loss: 40.8961 Acc: 0.5902
val Loss: 20.9824 Acc: 0.7582

Epoch: 1/19
----------
train Loss: 33.6060 Acc: 0.7500
val Loss: 17.7999 Acc: 0.8366

Epoch: 2/19
----------
train Loss: 33.0516 Acc: 0.7500
val Loss: 14.4657 Acc: 0.8889

Epoch: 3/19
----------
train Loss: 28.4793 Acc: 0.8156
val Loss: 12.3203 Acc: 0.9346

Epoch: 4/19
----------
train Loss: 28.8603 Acc: 0.7910
val Loss: 11.3680 Acc: 0.9281

Epoch: 5/19
----------
train Loss: 28.6729 Acc: 0.7500
val Loss: 10.0087 Acc: 0.9216

Epoch: 6/19
----------
train Loss: 26.2923 Acc: 0.8033
val Loss: 9.5608 Acc: 0.9412

Epoch: 7/19
----------
train Loss: 28.7745 Acc: 0.7828
val Loss: 9.2255 Acc: 0.9477

Epoch: 8/19
----------
train Loss: 25.5442 Acc: 0.8074
val Loss: 9.3479 Acc: 0.9346

Epoch: 9/19
----------
train Loss: 22.8637 Acc: 0.8484
val Loss: 9.4145 Acc: 0.9346

Epoch: 10/19
----------
train Loss: 25.1938 Acc: 0.8115
val Loss: 9.3248 Acc: 0.9477

Epoch: 11/19
----------
train Loss: 28.06

In [ ]:
# ======================================================================
# RINGKASAN PROGRAM (TRANSFER LEARNING CNN - RESNET18)
# ======================================================================
# 1. Dataset
#    - Dataset gambar dibagi menjadi TRAIN dan VALIDATION
#    - Struktur folder mengikuti ImageFolder:
#      train/class_name/, val/class_name/
#    - Kelas otomatis dibaca dari nama folder

# 2. Preprocessing & Transformasi Data
#    - Menggunakan ukuran input 224x224 (standar ResNet)
#    - Train:
#        * RandomResizedCrop → augmentasi data
#        * RandomHorizontalFlip → meningkatkan generalisasi
#    - Validation:
#        * Resize + CenterCrop → konsisten (tanpa randomness)
#    - Normalisasi menggunakan mean & std ImageNet
#      (WAJIB untuk pretrained model)

# 3. DataLoader
#    - Batch size = 4
#    - Shuffle aktif untuk train dan val
#    - Bertugas mengatur batching dan loading data

# 4. Model
#    - Menggunakan ResNet18 pretrained (ImageNet)
#    - Semua parameter convolutional layer dibekukan
#      (feature extractor tidak dilatih ulang)
#    - Fully Connected (FC) layer terakhir diganti:
#        dari 1000 output → 2 output (jumlah kelas dataset)
#    - Hanya FC layer yang dilatih

# 5. Loss Function & Optimizer
#    - Loss: CrossEntropyLoss (klasifikasi multi-class)
#    - Optimizer: SGD
#    - Learning Rate Scheduler:
#        * StepLR → LR diturunkan setiap 7 epoch

# 6. Training Loop
#    - Setiap epoch terdiri dari:
#        * Phase train → forward + backward + update weight
#        * Phase val → forward saja (tanpa grad)
#    - Akurasi dan loss dihitung per epoch
#    - Model terbaik disimpan berdasarkan validation accuracy

# 7. Konsep Utama
#    - Arsitektur ini adalah CNN berbasis Transfer Learning
#    - CNN berfungsi sebagai feature extractor
#    - Fully Connected layer berfungsi sebagai classifier
#    - Cocok untuk dataset kecil dengan hasil akurasi tinggi
# ======================================================================
